# Load and Display Files Name and Sizes

In [0]:
from pyspark.sql.functions import col

# Define the Azure Blob Storage account details
account_name = "b2410saos" #"your_account_name"
container_name = "cdrive"#"your_container_name"
sas_token = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2025-12-02T22:21:23Z&st=2024-12-02T14:21:23Z&spr=https,http&sig=HFAGH0i5Fw1voajhIQt%2FXjUS3SFKMRiOouCRhij%2FPIE%3D" #"your_sas_token"

# Set the configuration for accessing Azure Blob Storage
spark.conf.set(f"fs.azure.sas.{container_name}.{account_name}.blob.core.windows.net", sas_token)

# List the files in the container
files_df = spark.read.format("binaryFile").load(f"wasbs://{container_name}@{account_name}.blob.core.windows.net/")

# Select the path and length (size) of each file
files_sizes_df = files_df.select(col("path"), col("length").alias("size"))

# Display the file sizes
display(files_sizes_df)

path,size
wasbs://cdrive@b2410saos.blob.core.windows.net/SavedQuery.txt,243


# Read files recursively from Azure Blob Storage and display their sizes

In [0]:
from pyspark.sql.functions import col

# Define the Azure Blob Storage account details
account_name = "b2410saos"
container_name = "cdrive"
sas_token = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2025-12-02T22:21:23Z&st=2024-12-02T14:21:23Z&spr=https,http&sig=HFAGH0i5Fw1voajhIQt%2FXjUS3SFKMRiOouCRhij%2FPIE%3D"

# Set the configuration for accessing Azure Blob Storage
spark.conf.set(f"fs.azure.sas.{container_name}.{account_name}.blob.core.windows.net", sas_token)

# List the files in the container recursively
files_df = spark.read.format("binaryFile").option("recursiveFileLookup", "true").load(f"wasbs://{container_name}@{account_name}.blob.core.windows.net/")

# Select the path and length (size) of each file
files_sizes_df = files_df.select(col("path"), col("length").alias("size"))

# Display the file sizes
display(files_sizes_df)

path,size
wasbs://cdrive@b2410saos.blob.core.windows.net/Personal/20516321.jpg,18412
wasbs://cdrive@b2410saos.blob.core.windows.net/input/Load_multiple_sheets_of_excel.xlsx,17092
wasbs://cdrive@b2410saos.blob.core.windows.net/erp/Financial/ap/emp.csv,693
wasbs://cdrive@b2410saos.blob.core.windows.net/input/emp.csv,659
wasbs://cdrive@b2410saos.blob.core.windows.net/erp/parameterfile.json,402
wasbs://cdrive@b2410saos.blob.core.windows.net/SavedQuery.txt,243
wasbs://cdrive@b2410saos.blob.core.windows.net/input/test/employee.csv,203
wasbs://cdrive@b2410saos.blob.core.windows.net/erp/Financial/ar/dept.csv,102
wasbs://cdrive@b2410saos.blob.core.windows.net/program Files/documenttools/zoomit/dept.csv,102
wasbs://cdrive@b2410saos.blob.core.windows.net/input/24/10/file1.csv,44


# Delete files older than 2 days and size more than 200 bytes from Azure Blob Storage

In [0]:
from pyspark.sql.functions import col, unix_timestamp, current_timestamp,expr

# Define the Azure Blob Storage account details
account_name = "b2410saos"
container_name = "cdrive"
sas_token = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2025-12-02T22:21:23Z&st=2024-12-02T14:21:23Z&spr=https,http&sig=HFAGH0i5Fw1voajhIQt%2FXjUS3SFKMRiOouCRhij%2FPIE%3D"

# Set the configuration for accessing Azure Blob Storage
spark.conf.set(f"fs.azure.sas.{container_name}.{account_name}.blob.core.windows.net", sas_token)

# List the files in the container recursively
files_df = spark.read.format("binaryFile").option("recursiveFileLookup", "true").load(f"wasbs://{container_name}@{account_name}.blob.core.windows.net/")

# Filter files older than 7 days and size more than 200 bytes
seven_days_ago = (current_timestamp() - expr("INTERVAL 2 DAYS")).cast("timestamp")

filtered_files_df = files_df.filter((col("modificationTime") < seven_days_ago) & (col("length") > 200))

# Select the path of each file to be deleted
files_to_delete_df = filtered_files_df.select(col("path"))

# Collect the paths to delete
files_to_delete = [row.path for row in files_to_delete_df.collect()]

# Delete the files
for file_path in files_to_delete:
    dbutils.fs.rm(file_path, True)

# Display the paths of deleted files
# display(files_to_delete_df)


path
